In [1]:
import pandas as pd 
import numpy as np

import folium
import requests
from geopy.geocoders import Nominatim


In [2]:

# url = 'http://api.data.go.kr/openapi/tn_pubr_public_elesch_mskul_lc_api'
# params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '100', 'type' : 'xml', 'schoolId' : '', 'schoolNm' : '', 'schoolSe' : '', 'fondDate' : '', 'fondType' : '', 'bnhhSe' : '', 'operSttus' : '', 'lnmadr' : '', 'rdnmadr' : '', 'cddcCode' : '', 'cddcNm' : '', 'edcSport' : '', 'edcSportNm' : '', 'creatDate' : '', 'changeDate' : '', 'latitude' : '', 'longitude' : '', 'referenceDate' : '', 'instt_code' : '' }

# response = requests.get(url, params=params)
# print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>30</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>\r\n'


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.info()

/tmp/ipykernel_426208/1266426744.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("train.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [4]:
# train.head()
# train.tail()
print(train.columns)
print(test.columns)

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')
Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과

In [5]:
coord = train[["좌표X", '좌표Y']]
print(coord.isnull().sum())
# coord_new = np.array(coord)
# print(coord.iloc[975])
# coord = coord.replace(' ', np.nan)
lon = coord['좌표X'].unique()
lat = coord['좌표Y'].unique()
coord_new = pd.DataFrame({'좌표X':lon,
                          '좌표Y':lat})
coord_new.dropna(inplace=True)


좌표X    869670
좌표Y    869670
dtype: int64


In [6]:
centre = [37.541, 126.986]

map = folium.Map(location=centre, zoom_start = 10, tiles="Cartodb Positron")

for _, row  in coord_new.iterrows() :
    folium.CircleMarker(location=[row['좌표Y'], row['좌표X']], radius=2).add_to(map)
    # print(row['좌표Y'])

map

In [7]:
coord_apt = train[["아파트명", "좌표X", "좌표Y"]]
coord_apt = coord_apt.dropna()
print(len(coord_apt["아파트명"].unique()))
print(len(coord_apt["좌표X"].unique()))
print(len(coord_apt["좌표Y"].unique()))

unique_idx = coord_apt.drop_duplicates(subset="아파트명", keep="first").index
coord_apt_new = coord_apt.loc[unique_idx]


734
741
741


In [8]:
# centre = [37.541, 126.986]

# map = folium.Map(location=centre, zoom_start = 10, tiles="Cartodb Positron")

# for _, row  in coord_apt_new.iterrows() :

#     popup = folium.Popup(row['아파트명'], max_width=600)
#     folium.CircleMarker(location=[row['좌표Y'], row['좌표X']], popup=popup, radius=5).add_to(map)

#     # print(row['좌표Y'])

# map

In [9]:
train_new = train[["아파트명", "좌표X", "좌표Y", "시군구", "번지"]]
apt_name = train_new["아파트명"].unique()

print(train_new.iloc[0])
geolocator = Nominatim(user_agent="geoapi")
location_test = geolocator.geocode("서울시 강남구 개포동 658-1")
# print(location_test.latitude)
# print(location_test.longitude)

train_new["주소명"] = train["시군구"] + ' ' + train["번지"]

아파트명           개포6차우성
좌표X         127.05721
좌표Y         37.476763
시군구     서울특별시 강남구 개포동
번지              658-1
Name: 0, dtype: object


/tmp/ipykernel_426208/1402211739.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_new["주소명"] = train["시군구"] + ' ' + train["번지"]


In [29]:
geolocator = Nominatim(user_agent="geoapi")

def get_coordinates(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return None, None


sample = train_new[0:3]
# new_coor = []
new_x = []; new_y = []

for i in range(3):

    temp_coor = get_coordinates(sample["주소명"][i])
    new_x.append(temp_coor[0])
    new_y.append(temp_coor[1])
    

In [10]:
schools = pd.read_csv('schools.csv', encoding='cp949')
schools_new = schools[schools["소재지지번주소"].str.contains('서울')]
# schools_new.head()
print(len(schools_new))

1309
